# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper`.

You will find more info about virtual environments [here](../2.python/1.python_fundamentals/02.Package-Managers/00.Package_Managers.md).

Activate it.

## API Scraping

### A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword.

NOTE: external libraries need to be installed first. Check their documentation.

Check the [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the Requests documentation to:
1. use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is the signification of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [4]:
# import the requests library (1 line)
import requests
# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
url = "https://country-leaders.onrender.com/"
# assign the /status endpoint to another variable called status_url (1 line)
status_url = "status"
# query the /status endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(url + status_url)
# check the status_code using a condition and print appropriate messages (4 lines)
if req.status_code == 200:
    print(req.content)
else:
    print(f"A problem has occured during the request to {url}/{status_url}!")


b'"Alive"'


### Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. Check what to do in the Requests [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/).


In [5]:
# Set the country_url variable (1 line)
country_url = "countries"
# query the /countries endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(url + country_url)
# Get the JSON content and store it in the countries variable (1 line)
countries = req.json()
# display the request's status code and the countries variable (1 line)
print(req.status_code, countries)

403 {'message': 'The cookie is missing'}


### Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [6]:
# Set the cookie_url variable (1 line)
cookie_url = "cookie"
# Query the enpoint, set the cookies variable and display it (3 lines)
cookies = requests.get(url + cookie_url)

Try to query the countries endpoint using the cookie, save the output and print it.

In [7]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
countries = requests.get(url + country_url)
# and check the status code and the countries variable's content (3 lines)
if countries.status_code == 200:
    print(countries.json())

In [8]:
cookies = requests.get(url + cookie_url)
countries = requests.get(url + country_url, cookies= cookies.cookies)
if countries.status_code == 200:
    print(countries.json())

['us', 'ma', 'be', 'fr', 'ru']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### Getting the actual data from the API

Query the `/leaders` endpoint.

In [9]:
# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders_url = "leaders"
cookies = requests.get(url + cookie_url)
leaders = requests.get(url + leaders_url,  cookies= cookies.cookies)
# and check its content (4 lines)
if leaders.status_code == 200:
    print(leaders.json())
else:
    print(leaders.status_code)

404


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [10]:
# Set the leaders_url variable (1 line)

# query the /leaders endpoint using cookies and parameters (take any country in countries)


leaders = []
for elem in countries.json():
    cookies = requests.get(url + cookie_url)
    leaders.append(requests.get(url + leaders_url, 
                           params= {f"country": {elem}},
                           cookies= cookies.cookies).json())
# check the result (3 lines) 
for leader in leaders:
    for l in leader:
        test = l['wikipedia_url']
        print(test)

https://en.wikipedia.org/wiki/George_Washington
https://en.wikipedia.org/wiki/Barack_Obama
https://en.wikipedia.org/wiki/Abraham_Lincoln
https://en.wikipedia.org/wiki/George_W._Bush
https://en.wikipedia.org/wiki/Bill_Clinton
https://en.wikipedia.org/wiki/Joe_Biden
https://en.wikipedia.org/wiki/Franklin_D._Roosevelt
https://en.wikipedia.org/wiki/Andrew_Johnson
https://en.wikipedia.org/wiki/Gerald_Ford
https://en.wikipedia.org/wiki/Richard_Nixon
https://en.wikipedia.org/wiki/Lyndon_B._Johnson
https://en.wikipedia.org/wiki/John_F._Kennedy
https://en.wikipedia.org/wiki/Dwight_D._Eisenhower
https://en.wikipedia.org/wiki/Ronald_Reagan
https://en.wikipedia.org/wiki/Harry_S._Truman
https://en.wikipedia.org/wiki/John_Adams
https://en.wikipedia.org/wiki/Thomas_Jefferson
https://en.wikipedia.org/wiki/James_Madison
https://en.wikipedia.org/wiki/James_Monroe
https://en.wikipedia.org/wiki/John_Quincy_Adams
https://en.wikipedia.org/wiki/Andrew_Jackson
https://en.wikipedia.org/wiki/Martin_Van_Buren
ht

### A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each entry in a dictionary called `leaders_per_country` like this one: 
`{'fr':[{leader0}, {leader1},...],
'en':[{leader0}, {leader1},...],
...
}`.

In [ ]:
# 5 lines


It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [ ]:
# 15 lines


Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [ ]:
# 2 lines


## Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [15]:
# 3 lines
import requests
macron_url = 'https://fr.wikipedia.org/wiki/Emmanuel_Macron'
putin_url = "https://ru.wikipedia.org/wiki/%D0%9F%D1%83%D1%82%D0%B8%D0%BD,_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B8%D1%87"
req = requests.get("https://ru.wikipedia.org/wiki/%D0%9F%D1%83%D1%82%D0%B8%D0%BD,_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B8%D1%87")

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function to take a look. You will start the actual parsing in the next step.

In [17]:
# 3 lines
import bs4


In [18]:
def get_first_paragraph(url):
    req = requests.get(url)
    soup = bs4.BeautifulSoup(req.text, "html.parser")
    for paragraph in soup.find_all('p'):
        if paragraph.find_all('b'):
            return paragraph.text

In [19]:
get_first_paragraph(url=macron_url)

"Emmanuel Macron (/ɛmanɥɛl makʁɔ̃/[e] Écouter), né le 21 décembre 1977 à Amiens est un haut fonctionnaire et homme d'État français. Il est président de la République française depuis 2017.\n"

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [ ]:
# 2 lines


If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one
  * store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
  * exit the loop

At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [20]:
# 10 lines
# def get_first_paragraph(wikipedia_url):
#   print(wikipedia_url) # keep this for the rest of the notebook
#   [insert your code]
#   return first_paragraph
def get_first_paragraph(url):
    print(f"Getting the first paragraph of : {url}")
    req = requests.get(url)
    soup = bs4.BeautifulSoup(req.text, "html.parser")
    for paragraph in soup.find_all('p'):
        if paragraph.find_all('b'):
            return paragraph.text

putin_leader = get_first_paragraph(putin_url)
macron_leader = get_first_paragraph(macron_url)
print(putin_leader)
print("\n")
print(macron_leader)
# 6 lines


Getting the first paragraph of : https://ru.wikipedia.org/wiki/%D0%9F%D1%83%D1%82%D0%B8%D0%BD,_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B8%D1%87
Getting the first paragraph of : https://fr.wikipedia.org/wiki/Emmanuel_Macron
Влади́мир Влади́мирович Пу́тин (род. 7 октября 1952, Ленинград, СССР) — российский государственный и политический деятель. Действующий президент Российской Федерации, председатель Государственного Совета Российской Федерации и верховный главнокомандующий Вооружёнными силами Российской Федерации с 7 мая 2012 года. Ранее занимал должность президента с 7 мая 2000 года по 7 мая 2008 года, также в 1999—2000 и 2008—2012 годах занимал должность председателя правительства Российской Федерации. Кандидат экономических наук (1997).



Emmanuel Macron (/ɛmanɥɛl makʁɔ̃/[e] Écouter), né le 21 décembre 1977 à Amiens est un haut fonctionnaire et homme d'État français. Il est président de la République française

### Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [21]:
# 4 lines


import re
macron_leader = re.sub(" \([^()]+\)", '',macron_leader)
macron_leader = re.sub("(?Phttps?://[^\s]+)", '', macron_leader)
print(macron_leader)



error: unknown extension ?Ph at position 1

Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [22]:
# 10 lines
def get_first_paragraph(url):
    print(f"Getting the first paragraph of : {url}")
    req = requests.get(url)
    soup = bs4.BeautifulSoup(req.text, "html.parser")
    for paragraph in soup.find_all('p'):
        if paragraph.find_all('b'):
            first_paragraph = re.sub(" \([^()]+\)", '', paragraph.text)
            first_paragraph = re.sub("(?Phttps?://[^\s]+)", '', first_paragraph)
            return first_paragraph

putin_leader = get_first_paragraph(putin_url)
macron_leader = get_first_paragraph(macron_url)
print(putin_leader)
print("\n")
print(macron_leader)

Getting the first paragraph of : https://ru.wikipedia.org/wiki/%D0%9F%D1%83%D1%82%D0%B8%D0%BD,_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B8%D1%87


error: unknown extension ?Ph at position 1

Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [ ]:
# < 20 lines


### Caching
Good job! This function will be called many times. First because there are many leaders to loop through. Second because you are developing your code and are likely to re-run your loop multiple times. Third because the API might change and contain new leaders and you might want to update your data periodically.

For all these reasons, it can be benificial to *cache* the result of the `get_first_paragraph()` function so that if it called with the same parameter, you expect the same output and thus do not need to query Wikipedia another time.

You will use the [lru_cache](https://docs.python.org/3/library/functools.html#functools.lru_cache) decorator from the `functools` module. It is a tiny code change that will speed things up big time. You can use a `maxsize` of None. If you have questions about decorators, you will find more information [here](../../2.python/2.python_advanced/07.Decorator/decorators.ipynb)

In [ ]:
# < 20 lines
from functools import lru_cache

@lru_cache(maxsize = None)
def get_first_paragraph(url):
    print(f"Getting the first paragraph of : {url}")
    req = requests.get(url)
    soup = bs4.BeautifulSoup(req.text, "html.parser")
    for paragraph in soup.find_all('p'):
        if paragraph.find_all('b'):
            first_paragraph = re.sub(" \([^()]+\)", '', paragraph.text)
            first_paragraph = re.sub("(?Phttps?://[^\s]+)", '', first_paragraph)
            return first_paragraph

Let's check the speedup!

In [ ]:
# Just run the cell. 
%%timeit
get_first_paragraph(wikipedia_url)

## Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [ ]:
# 19 lines
@lru_cache(maxsize = None)
def get_first_paragraph(url):
    """Get the first paragraph of the wikipedia link given to it"""
    try:
        req = requests.get(url)
        if req.status_code != 200:
            raise Exception(f"Error : Unable to access {url}")
    except Exception as e:
        print(e)
        return None
    
    soup = bs4.BeautifulSoup(req.text, "html.parser")
    for paragraph in soup.find_all('p'):
        if paragraph.find_all('b'):
            #removing parenthese and what is between them
            first_paragraph = re.sub(" \([^()]+\)", '', paragraph.text)
            #removing url link
            first_paragraph = re.sub("(?Phttps?://[^\s]+)", '', first_paragraph)
            return first_paragraph
        else:
            return None

def get_leaders():
    """Get all the leaders of the five contries given by the API"""
    with Session() as session:
    url = "https://country-leaders.onrender.com/"
    country_url = "countries"
    cookie_url = "cookie"
    leaders_url = "leaders"
    leaders = []
    firsts_pragraphe = []

  try:
        cookie = requests.get(url + cookie_url)
        countries = requests.get(url + country_url, 
                             cookies= cookie.cookies)
        if countries.status_code != 200:
            raise Exception("Error : Unable to get the countries name !")
    except Exception as e:
        print(e)
        return None
    countries = countries.json()
    
    for country in countries:
        try:
            cookie = requests.get(url + cookie_url)
            leader = requests.get(url + leaders_url,
                                    params= {f"country": {country}},
                                    cookies= cookie.cookies)
            if leader.status_code != 200:
                raise Exception(f"Error : Unable to get the leader of {country}")
            else:
                for l in leader.json():
                   firsts_pragraphe.append(get_first_paragraph(l['wikipedia_url']))
        except Exception as e:
            print(e)
            leader = None
        leaders.append(leader)
        
    leaders_per_country = {countries[0]: leaders[0],
                           countries[1]: leaders[1],
                           countries[2]: leaders[2],
                           countries[3]: leaders[3],
                           countries[4]: leaders[4]}
    for paragraphe in firsts_pragraphe:
        print(paragraphe)
    return leaders_per_country

leaders_per_coutry = get_leaders()

In [ ]:
# Check the output of your function (2 lines)


Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function to check if the `status_code` is a cookie error. In which case, get new ones and query the api again.

If not, you may have a fast connection or a bit of luck. Do it anyway.

In [ ]:
# 22 lines


Check the output of your function again.

In [ ]:
%%timeit
# Check the output of your function (1 line)


Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [ ]:
# <20 lines

Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [ ]:
# 22 lines


Test your new functions.



In [ ]:
# Just run it.
%%timeit
leaders_per_country = get_leaders()

Using a Session makes the first call to `get_leaders()` faster however, for technical reasons, the `@lru_cache` decorator no longer works. 

Problem: The `session` parameter is not *hashable* and since the cache is a dictionary using all the parameters as key, this poses a problem.

Solution: Create our own cache that only cares about the url.

The code for this is provided. You should only read it.

In [ ]:
cache = {}
def hashable_cache(f):
    def inner(url, session):
        if url not in cache:
            cache[url] = f(url, session)
        return cache[url]
    return inner

Overwrite the `get_first_paragraph()` function to use the new decorator like this: `@hashable_cache`.


In [ ]:
# <20 lines

Check that the cache is working again.

In [ ]:
# Just run it.
%%timeit
leaders_per_country = get_leaders()

## Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Make a tour in our [file handling section](../../2.python/2.python_advanced/04.File-handling/file_handling.ipynb) to know more about how to read and write files using Python.

In [ ]:
# 3 lines


Make sure the file can be read back. Write the code to read the file.

In [ ]:
#3 lines

Make a function `save(leaders_per_country)` to call this code easily.

In [ ]:
# 3 lines


In [ ]:
# Call the function (1 line)

## Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## To go further

If you want to practice scraping, you can read this section and tackle the exercise.

You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!